In [1]:
import os
import nltk
import cv2
import pandas as pd
import math
import numpy as np
import warnings
warnings.filterwarnings("ignore")

train_file = "train_data.csv"
test_file = "test_data.csv"

def load_data(file, direc="", sep=",", header=True):
    csv_path = os.path.join(direc, file)
    if header:
        return pd.read_csv(csv_path, sep=sep, index_col=False)
    else:
        return pd.read_csv(csv_path, sep=sep, index_col=False, header=None)
    

In [2]:
train_data = load_data(train_file)

In [3]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,280,281,282,283,284,285,286,287,288,Sentiment
0,-0.190214,-0.775909,0.606918,-0.422152,1.006142,-1.099935,-0.591228,0.110556,-0.685102,0.233484,...,-0.406460,-0.145558,-1.306575,0.230331,0.624839,-1.368549,0.805194,-0.758950,0.145932,1.0
1,-0.248992,-0.204263,0.585718,-0.340346,0.623218,-0.132182,-0.446606,-0.085173,0.247781,-0.290398,...,-0.172685,0.630882,0.110635,0.223473,0.180395,0.131066,0.163105,0.723476,-0.002338,0.0
2,0.040464,1.032194,-0.982082,-0.054039,0.172879,0.061846,0.513306,-0.427153,0.800728,-0.842768,...,-0.071184,0.454823,-0.388393,-0.455052,0.816787,-0.742167,0.423050,0.541922,-0.391158,1.0
3,-0.102443,-0.471759,0.036330,0.192056,1.049041,0.023328,-0.082436,-0.192707,-0.043875,0.224525,...,0.502373,-0.552040,-0.389139,0.310724,-0.114863,0.532303,-0.150409,0.128998,-0.101038,1.0
4,0.029157,-0.296443,0.683454,0.026448,-0.729840,1.183778,0.836208,0.700741,-0.260888,-0.136369,...,-0.318998,-0.120458,0.013771,-0.661265,-0.215869,-1.508247,0.381819,-0.027416,-0.900369,0.0


In [4]:
test_data = load_data(test_file)

In [5]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,280,281,282,283,284,285,286,287,288,Sentiment
0,-0.224514,-0.269337,0.149535,-0.204498,-0.319069,-0.178723,0.249171,0.177335,-0.430412,-0.079033,...,0.477580,0.326845,0.422710,-0.071163,-0.674291,0.361922,-0.018685,-0.334325,0.639806,0.0
1,-0.428177,0.066324,0.035085,-0.386542,-0.237192,0.059233,-0.268529,0.356585,0.095090,-0.367670,...,-0.086699,0.465709,-0.067263,-0.474439,-0.090253,-0.149597,-0.047571,-0.148241,-0.014998,1.0
2,0.000866,-0.354202,-0.428207,-0.127960,0.188175,-0.415807,0.359235,0.047495,0.167235,-0.148469,...,0.468923,-0.178838,0.158628,0.260055,0.042674,-0.051289,0.167824,-0.244007,-0.143949,0.0
3,0.094622,-0.015890,0.131040,0.235522,0.344240,0.094459,-0.260801,-0.167188,-0.103057,0.209749,...,-0.128503,-0.116131,-0.026830,-0.217328,0.366705,-0.037544,-0.442792,0.306934,0.205799,0.0
4,0.419940,0.225918,0.188928,0.090584,-0.360560,-0.180346,0.190001,0.260009,0.066459,0.268290,...,0.014408,0.333524,-0.011516,-0.129948,-0.256695,-0.075641,0.134001,-0.198112,-0.013948,0.0


In [6]:
train_labels = np.int16(train_data["Sentiment"].copy().values)
train_features = train_data.drop("Sentiment", axis=1)

test_labels = np.int16(test_data["Sentiment"].copy().values)
test_features = test_data.drop("Sentiment", axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
scalar.fit(train_features)

train_features = scalar.transform(train_features)
test_features = scalar.transform(test_features)

In [8]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
def plot_roc_curve(clf_sets):
    for clf_set in clf_sets:
        y = clf_set[0]
        y_pred = clf_set[1]
        label = clf_set[2]
        fpr, tpr, thresholds = roc_curve(y, y_pred)
        plt.plot(fpr, tpr, linewidth=1, label=label)
    
    plt.plot([0,1],[0,1],'k--')
    plt.axis([0,1,0,1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="bottom right")
    plt.show()

In [9]:
X = train_features.copy()
Y = train_labels.copy()
X_test = test_features.copy()
Y_test = test_labels.copy()

In [ ]:

# CNN Classifier
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

batch_size = 64
epochs = 25
TRAIN_MODEL = True
MODEL_NAME = "trained_model.h5"

size = np.int16(np.sqrt(X.shape[1]))

train_x = np.reshape(X, (-1, size, size, 1))
test_x = np.reshape(X_test, (-1, size, size, 1))

#binarizer = LabelBinarizer()
#binarizer.fit(Y)
#train_y = binarizer.transform(Y)
#test_y = binarizer.transform(Y_test)

train_y = Y
test_y = Y_test

num_classes = 1 #len(binarizer.classes_)
print(num_classes)
droprate = 0.7

try:
    model = load_model(MODEL_NAME)
except:
    model = None

if model is None:
    model = Sequential()
    model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='elu', input_shape=(size, size, 1)))
    model.add(BatchNormalization())

    model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='elu', padding='valid'))
    model.add(BatchNormalization())

    model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='elu', padding='valid'))
    model.add(BatchNormalization())

    model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='elu', padding='valid'))
    model.add(BatchNormalization())
    
    model.add(Dropout(droprate))
    model.add(Flatten())

    model.add(Dense(256, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(droprate))

    model.add(Dense(128, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(droprate))

    model.add(Dense(64, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(droprate))
    
    model.add(Dense(32, activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(droprate))

    model.add(Dense(num_classes, activation='sigmoid'))

else:
    print(MODEL_NAME, " is restored.")

model.summary()

adam = Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

callbacks = [ModelCheckpoint(MODEL_NAME, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)]

if TRAIN_MODEL:
    history = model.fit(train_x, train_y,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(test_x, test_y),
                        callbacks=callbacks)
else:
    print("Opted not to train the model as TRAIN_MODEL is set to False. May be because model is already trained and is now being used for validation")
    


In [ ]:
saved_model = load_model(MODEL_NAME)
score = saved_model.evaluate(test_x, test_y, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])